In [ ]:
!pip install -qU langchain==0.1.0 langchain-google-genai==0.0.9 langchain-community==0.0.19 python-dotenv

In [ ]:
import os
#from google.colab import userdata
from langchain.agents import initialize_agent, AgentType, Tool
from langchain_google_genai import ChatGoogleGenerativeAI

# 🔑 SETUP API KEY
os.environ["GOOGLE_API_KEY"] = ""
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1,convert_system_message_to_human=True)

print("✅ Libraries installed and LLM ready.")

In [ ]:
# Centralized Archiecture : A single agent has access to multiple tools

# Step 1 : Define the tools [workers]
def product_search(query):
  "Worker 1 : the product expert"
  return "Found: iphone 15 (80,000 INR), Samsung S24 (75000 INR), all in stock."

def policy_search(query):
  "Worker 2 : The policy expert"
  return "Policy : 7-day return for phones, no returns for accessories."

# Step 2 : Wrap them as langchain tools!
product_tool = Tool(
    name = "product_search",
    func=product_search,
    description="useful for when you want to know price, stock"
)

policy_tool = Tool(
    name = "policy_search",
    func=policy_search,
    description="useful for when you want to know return policy"
)

In [ ]:
# Step 3 : Define the manager / orchestrator
def setup_centralized_agent():
  tools = [product_tool,policy_tool]

  agent = initialize_agent(tools=tools,
                           llm=llm,
                           verbose=True,
                           agent = AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION)
  return agent

In [ ]:
# Step 4 :
manager = setup_centralized_agent()

In [ ]:
manager.run("Whats the price of iphone 15?")

In [ ]:
manager.run("Whats the return on accessories?")

In [ ]:
manager.run("Whats the price of iphone 15 and return policy??")

In [ ]:
# ==========================================
# CELL 3: DECENTRALIZED ARCHITECTURE (The Hand-off)
# ==========================================
from langchain.agents import initialize_agent, AgentType, Tool

# --- Step 1: Create Individual Agents ---
# We use ZERO_SHOT_REACT_DESCRIPTION because it handles simple string inputs better.

product_agent = initialize_agent(
    tools=[product_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # <--- CHANGED THIS
    verbose=True,
    handle_parsing_errors=True # Helps agent recover if it messes up formatting
)

policy_agent = initialize_agent(
    tools=[policy_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # <--- CHANGED THIS
    verbose=True,
    handle_parsing_errors=True
)

# --- Step 2: The Hand-off Logic (The "Glue") ---
def run_decentralized_flow(query):
    print(f"User Query: {query}")
    print("👉 Sending to Product Agent first...")

    try:
        # 1. Attempt with Product Agent
        response = product_agent.run(query)

        # 2. SIMULATED CHECK: The agent might answer "I don't know", but for
        # this demo, we force a hard failure if it's the wrong domain.
        if "return" in query.lower() or "policy" in query.lower():
            raise ValueError("Not my expertise!")

        return response

    except Exception as e:
        # 3. The Fallback (Hand-off)
        print(f"\n❌ Product Agent refused: {e}")
        print("🔄 Handing off to Policy Agent...")
        return policy_agent.run(query)

# --- Step 3: Run It ---
print("\n🤖 DECENTRALIZED SYSTEM: Passing the baton.")
final_answer = run_decentralized_flow("What is the return policy?")
print(f"\n📢 Final Answer: {final_answer}")

In [ ]:
# Hybrid Architecture (The Smart Router)
# LLM decide - who is the best to answer this question?

# Step 1 :
def smart_router_and_execute(query):
  router_prompt = f"""
  Classify this query into exactly 1 category : PRODUCT, POLICY or GENERAL.
  Query : {query}
  Rules:
  - PRODUCT: Buying, prices, stock, device specs.
  - POLICY: Returns, refunds, warranty, shipping, support.
  - GENERAL: Greetings, how are you, general help.

  Return ONLY the category word.
"""
  category = llm.invoke(router_prompt).content.strip().upper()


  # Step 2 : Dispatching for execution
  # we re-use the agents we created in the step above!
  if "PRODUCT" in category :
    return product_agent.run(query)

  elif "POLICY" in category :
    return policy_agent.run(query)

  else :
    return "Hi, please ask questions only about policies / prices."

In [ ]:
smart_router_and_execute("I want to buy a Samsung phone, how much is it?")

In [ ]:
def smart_router_and_execute(query):
  router_prompt = f"""
  Classify this query into exactly 1 category : PRODUCT, POLICY or GENERAL.
  Query : {query}
  Rules:
  - PRODUCT: Buying, prices, stock, device specs.
  - POLICY: Returns, refunds, warranty, shipping, support.
  - GENERAL: Greetings, how are you, general help.

  Return ONLY the category word.
"""
  category = llm.invoke(router_prompt).content.strip().upper()
  print(category)

In [ ]:
smart_router_and_execute("My iphone is broken, i need to return it and I want to buy a Samsung phone, how much is it?")